# QOSF Mentorship Program: Screening Task 4

### The task is to find the lowest eigenvalue of a given $H$ matrix using Variational Quantum Eigensolver where, 

$$
H = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & 0 & 1\\
\end{pmatrix}
$$

The first step is to decompose the Hamiltionian $H$ matrix as a linear combination of Pauli matrices. Here 2 qubit pauli matrices are used since $H$ is a 4x4 matrix. It should be noted that $XX \equiv X \otimes X$, and the notation is used only for convenience. The pauli matrices are given as: 

$$
XX = \begin{pmatrix}
0 & 0 & 0 & 1\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
1 & 0 & 0 & 0\\
\end{pmatrix},

\qquad

YY = \begin{pmatrix}
0 & 0 & 0 & -1\\
0 & 0 & 1 & 0\\
0 & 1 & 0 & 0\\
-1 & 0 & 0 & 0\\
\end{pmatrix}, 

\qquad

ZZ = \begin{pmatrix}
1 & 0 & 0 & 0\\
0 & -1 & 0 & 0\\
0 & 0 & -1 & 0\\
0 & 0 & 0 & 1\\
\end{pmatrix}.
$$

Now, its very intuitive that simply adding XX and YY gives us the twice of the rows 2 and 3.
$$
XX+YY = \begin{pmatrix}
0 & 0 & 0 & 0\\
0 & 0 & 2 & 0\\
0 & 2 & 0 & 0\\
0 & 0 & 0 & 0\\
\end{pmatrix}
$$


Similarly, adding ZZ and II gives us ttwice of the first and last rows.

$$
ZZ + II = \begin{pmatrix}
2 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 2\\
\end{pmatrix}
$$

Its very clear from here that $H = \frac{1}{2} (ZZ + II - XX - YY)$. 

In order to create separate ciruits for each term, we can write $H = H_{1} + H_{2} + H_{3} + H_{4}$, where 

$$

H_{1} = \frac{1}{2} ZZ,\quad

H_{2} = -\frac{1}{2} XX,\quad

H_{3} = -\frac{1}{2} YY,\quad

H_{4} = \frac{1}{2} II.\quad
$$


In [1]:
from qiskit import *
from qiskit import Aer, execute
import numpy as np
from numpy import pi

### This program uses QISKIT statevector simulator backend since we want to find the lowest theoretical eigenvalue.

In [2]:
backend = Aer.get_backend('statevector_simulator')
# The number of measurements to implement
num_shots = 1000                                  

### The next cell defines a function to find the probability of each state by squaring the absolute value of amplitude in a given statevector.

In [3]:
def get_prob(v):
    return abs(v)**2

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
$$

## Expectation values of 2 qubit Pauli matrices

### The following cell defines a function to find the expectation value of the $H$ matrix by using the probability counts for each of the standard (Z) basis states.

### Let $ \ket{\psi} = \alpha \ket{00} + \beta \ket{01} + \gamma \ket{10} + \delta \ket{11} .$

### $ Z \ket{0} = \ket{0}, Z \ket{1} = - \ket{1} $

### $ \implies ZZ \ket{\psi} = \alpha \ket{00} - \beta \ket{01} - \gamma \ket{10} + \delta \ket{11} $

### Then $\bra{\psi} ZZ \ket{\psi} = \alpha^{2} - \beta^{2} - \gamma^{2} + \delta^{2} $

We know that the square of amplitude of an eigenstate corresponds to the probability of measuring it.

## $ \implies \bra{\psi} ZZ \ket{\psi} = P_{00} - P_{01} - P_{10} + P_{11} $

In [4]:
def expect(prob_vector): #(counts['00'] - counts['01'] - counts['10'] + counts['11'])/num_shots
    '''
    Input: The statevector probabilities.
    Output: Measurement probabilities in Z basis, with proability counts for each state added properly
    '''
    return (prob_vector[0] - prob_vector[1] - prob_vector[2] + prob_vector[3])

## Transformations for measuring XX and YY in the standard Z basis

In order to measure terms with X and Y pauli matrices in the standard $Z$ basis, some operation $U$ needs to be applied onto the original state $\ket{\psi}$.

For the $Z$ pauli matrix, no operation is needed to measure in $Z$ basis. Now, since $X$ and $Y$ basis are just rotations in the bloch sphere, we can convert these into the $Z$ basis by applying corresponding $RX(\theta)$ and $RY(\theta)$ gates to the circuit. If there is a $X$ pauli matrix, we undo it by applying the $RY(-\frac{\pi}{2})$, which is also the Hadamard gate and for a $Y$ pauli matrix, we apply the $RX(\frac{\pi}{2})$ gate. After applying these operations, the circuit can be measured in the $Z$ basis as usual.

### **Note:** The last term $H_{4}$ with the identity matrix doesn't need a circuit. This is because it is just a constant, and we can just add 1 later to the expectation values of the rest of the terms.

## Ansatz initial state for the wavefunction $\ket{\psi}$

Here the **ansatz** used for the **initial state** is: 

## $ \ket{\psi} = (RX(\theta) \otimes I) \,\, CX \,\, (H \otimes I)\,\, \ket{00} $, where $\theta$ is the variational parameter.

In [5]:
# Sample of the ansatz circuit
circ = QuantumCircuit(2, 2)
circ.rx(pi, 0)
circ.cx(0, 1)
circ.h(0)
circ.draw()
#circ.draw(output='mpl')

┌────────┐     ┌───┐
q_0: ┤ RX(pi) ├──■──┤ H ├
     └────────┘┌─┴─┐└───┘
q_1: ──────────┤ X ├─────
               └───┘     
c: 2/════════════════════

### The following cells define functions that create appropriate circuits for each term in the Hamiltonian matrix. 

In [6]:
def H1(theta):                      # ZZ term
    
    circ = QuantumCircuit(2, 2)     # Initialize 2-qubit Quantum Circuit

    # ansatz
    circ.rx(theta, 0)
    circ.cx(0, 1)
    circ.h(0)
    
    circ.measure([0,1], [0,1])

    result = execute(circ, backend, shots = num_shots).result()
    statevector = result.get_statevector(circ)
    return expect(get_prob(statevector))

In [7]:
def H2(theta):                      # XX term

    circ = QuantumCircuit(2, 2)     # Initialize 2-qubit Quantum Circuit
    
    # ansatz
    circ.rx(theta, 0)
    circ.cx(0, 1)
    circ.h(0)

    # Rotations to enable measurement in Z basis (equivalent alternative gate commente0d).
    circ.h(0)  #circ2.ry(-pi/2, 0)
    circ.h(1)  #circ2.ry(-pi/2, 1)

    circ.measure([0,1], [0,1])

    result = execute(circ, backend, shots = num_shots).result()
    statevector = result.get_statevector(circ)
    return expect(get_prob(statevector))

In [8]:
def H3(theta):                      # YY term

    circ = QuantumCircuit(2, 2)     # Initialize 2-qubit Quantum Circuit
    
    # ansatz
    circ.rx(theta, 0)
    circ.cx(0, 1)
    circ.h(0)

    # Rotations to enable measurement in Z basis
    circ.rx(pi/2, 0)
    circ.rx(pi/2, 1)
    
    circ.measure([0,1], [0,1])

    result = execute(circ, backend, shots = num_shots).result()
    statevector = result.get_statevector(circ)
    return expect(get_prob(statevector))

### The following cell defines a function which simply finds the expectation values of all the terms in the Hamiltonian, and returns half of that value.

In [9]:
def Hamiltonian_eigen(theta):
    c = H1(theta) - H2(theta) - H3(theta) + 1           # The 1 comes from the identity matrix in H4 term, since it always remains constant.
    return c/2                                          # The complete Hamiltonian is divided by 2, which is again just a constant scaling hence used here

## We simulate the code for 500 values (can be changed) of $\theta$ from 0 to 2$\pi$, and find the minimum eigenvalue using its outcomes.

In [10]:
intervals = 500
X = np.linspace(0, 2*pi, intervals)
Y = []
for i in X:
    Y.append(Hamiltonian_eigen(i))

In [11]:
predicted_eigenvalue = np.min(Y)
theta_value = X[Y.index(predicted_eigenvalue)]
print("="*60)
print("Lowest predicted eigenvalue is:", predicted_eigenvalue, "for theta =", theta_value)
print("="*60)

Lowest predicted eigenvalue is: -1.0 for theta = 0.025183107443605555


## Now let's check using numpy the actual lowest eigenvalue and verify our solution

In [12]:
H = [[1, 0, 0, 0], [0, 0, -1, 0], [0, -1, 0, 0], [0, 0, 0, 1]]
w, v = np.linalg.eig(H)
actual_eigenvalue = np.min(w)

prediction = "equal" if predicted_eigenvalue == actual_eigenvalue else "not equal"

print("="*60)
print("Actual lowest eigenvalue is:", actual_eigenvalue)
print("="*60)
print("The predicted lowest eigenvalue is", prediction, "to the actual eigenvalue.")

Actual lowest eigenvalue is: -1.0
The predicted lowest eigenvalue is equal to the actual eigenvalue.
